In [17]:
from IPython.display import display, HTML
display(HTML("<style>.jp-Cell { width: 200% !important; margin-left:-500px}</style>"))

In [23]:

# !pip install torch-geometric
# !pip install pytorch-lightning
# !pip install h5py
# !pip install matplotlib
# !pip install tensorboard

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import sys
sys.path.append('/gpfs/data/fs72150/springerd/Projects/LuttingerWard_from_ML/code/')
import torch
# from models import models as models
# from wrappers import wrapers
from torch.utils.data import DataLoader
import load_data
import datetime
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from pytorch_lightning.plugins.environments import LightningEnvironment
import json
import os
import numpy as np

def train():
    ### JSON File contains full information about entire run (model, data, hyperparameters)
    ### TODO 
    MODEL_NAME = "AUTO_ENCODER_1"
    config = json.load(open('../configs/confmod_auto_encoder.json'))[MODEL_NAME]
    # MODEL_NAME = "AUTO_ENCODER_1"
    # config = json.load(open('confmod_auto_encoder.json'))[MODEL_NAME]

    ''' Dataloading '''
    ### > Separate training and validation HDF5 files 
    # rand_samples = torch.randint(0,1000,(100,), dtype=int)
    # sample_idxs = torch.ones(100, dtype=int) * 941
    # sample_idx = torch.cat([sample_idxs, rand_samples], axis=0)
    # sample_idx, _ = torch.sort(sample_idx)
    # ld = __import__("load_data", fromlist=['object'])
    # train_set = getattr(ld, config["DATA_LOADER"])(config, data_type = "valid", target_sample = sample_idx)
    # # validation_set = getattr(ld, config["DATA_LOADER"])(config, data_type = "valid")
    # train_dataloader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True, num_workers=8, persistent_workers=True, pin_memory=True)
    # # validation_dataloader = DataLoader(validation_set, batch_size=config["batch_size"], shuffle=False, num_workers=8, persistent_workers=True, pin_memory=True)
    
    ### > Single HDF5 file containing training and validation data 
    ld = __import__("load_data", fromlist=['object'])
    # data_set = load_data.Dataset_ae(config)
    data_set = getattr(ld, config["DATA_LOADER"])(config, data_type = "insulating")
    train_set, validation_set = torch.utils.data.random_split(data_set, [int(data_set.__len__()*0.8), (data_set.__len__()-int(data_set.__len__()*0.8))], generator=torch.Generator().manual_seed(42))
    train_dataloader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True)
    validation_dataloader = DataLoader(validation_set, batch_size=config["batch_size"], shuffle=True)


    ''' Model setup '''
    wrapers = __import__("wrappers.wrapers", fromlist=['object'])#.wrapers
    # import sys, inspect
    # for name, clazz in inspect.getmembers(wrapers):
    #     print(name)
    #     if inspect.isclass(clazz):
    #         print(clazz)
    model = getattr(wrapers, config["MODEL_WRAPER"])(config)
    
    ''' Model loading from save file '''
    if config["continue"] == True:
        SAVEPATH = config["SAVEPATH"]
        checkpoint = torch.load(SAVEPATH)
        model.load_state_dict(checkpoint['state_dict'])
        print(" >>> Loaded checkpoint")


    ''' Logging and saving '''
    DATA_NAME = os.path.splitext(os.path.basename(config["PATH_TRAIN"]))[0]

    PATH = ""
    CONFIGURATION = f"../saves/{DATA_NAME}/save_{config['MODEL_NAME']}_BS{config['batch_size']}_{datetime.datetime.now().date()}"
    # CONFIGURATION = f"../saves/save_{config['MODEL_NAME']}_Nodes{config['n_nodes']}_BS{config['batch_size']}_{datetime.datetime.now().date()}"
    logger = TensorBoardLogger(PATH, name=CONFIGURATION)


    # '''Define (pytorch_lightning) Trainer '''
    # ### > SLURM Training
    # trainer = pl.Trainer(max_epochs=config["epochs"], accelerator=config["device_type"], devices=config["devices"], num_nodes=config["num_nodes"], strategy='ddp', logger=logger)
    # ### > Jupyter Notebook Training
    trainer = pl.Trainer(max_epochs=config["epochs"], accelerator=config["device_type"], devices=1, strategy='auto', logger=logger, plugins=[LightningEnvironment()])
    # ### > Jupyter Notebook CPU Training
    # trainer = pl.Trainer(max_epochs=1, accelerator='cpu', devices=1, strategy='auto', logger=logger, plugins=[LightningEnvironment()])
    
    ''' Train '''
    trainer.fit(model, train_dataloader, validation_dataloader)
    # trainer.fit(model, train_dataloader)
    
    ''' Saving configuration file into log folder ''' 
    LOGDIR = trainer.log_dir
    json_object = json.dumps(config, indent=4)
    with open(LOGDIR+"/config.json", "w") as outfile:
        outfile.write(json_object)




def main():
    train()

if __name__ == '__main__':
    main()
# %%


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type         | Params | Mode 
-------------------------------------------------------
0 | model         | auto_encoder | 307 K  | train
1 | criterion_mse | MSELoss      | 0      | train
-------------------------------------------------------
307 K     Trainable params
0         Non-trainable params
307 K     Total params
1.231     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
